In [1]:
# The purpose of this file is to check cluster stability and conduct sensitivity analysis #1: leave one cohort out
# created by Zhongyu Li, May 2024 
# Note: we will only use 6 cohort with HOMA2IR and HOMA2B data

In [2]:
# load libraries
import pandas as pd 
import numpy as np
import os as os

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler



In [6]:
os. getlogin()

'root'

In [ ]:
#not working, need to figure out why as of 5.18.24 

os. getlogin()

if os.getlogin()=="JVARGH7":
    path_folder = 'C:/Cloud/Emory University/li, zhongyu - Diabetes Endotypes Project (JV and ZL)'
if os.getlogin()=="zhongyuli":
    path_folder = '/Users/zhongyuli/Library/CloudStorage/OneDrive-EmoryUniversity/Diabetes Endotypes Project (JV and ZL)'

# Rename the dataset into analytic sample
analytic_dataset = pd.read_csv(path_folder + '/working/processed/final_dataset.csv')


In [3]:
# first, we will load the data and conduct k means clustering to create the "TRUE labels"
# we will use the HOMA2IR and HOMA2B data to create the labels
# we will use the first 6 cohorts


path_folder = '/Users/zhongyuli/Desktop/python/cluster analysis/dataset/final_dataset.csv' # remove after the above bug is fixed

analytic_dataset = pd.read_csv(path_folder) 

#drop missing values
method3_df = analytic_dataset[analytic_dataset['method3'] == 1]

#select variables 
selected_variables = ['bmi', 'hba1c', 'dmagediag','homa2b','homa2ir','tgl','ldlc','ratio_th','sbp','dbp','hdlc','study']

#drop missing values in the selected variables
method3_df = method3_df[selected_variables]
method3_df = method3_df.dropna()

#check the data
method3_df.head()



,bmi,hba1c,dmagediag,homa2b,homa2ir,tgl,ldlc,ratio_th,sbp,dbp,hdlc,study
0,33.37665,6.9,70.0,102.7,2.217295,124,83.0,3.179487,119.2454,65.9166,39.0,jhs
1,27.76165,6.5,68.0,113.6,1.945525,101,81.0,1.980392,119.2454,65.0865,51.0,jhs
2,30.73822,7.9,51.0,40.6,2.192982,117,131.0,3.342857,162.3397,99.1206,35.0,jhs
3,43.56290,6.9,65.0,99.9,2.469136,78,97.0,1.322034,130.2482,70.0671,59.0,jhs
4,29.33333,6.5,77.0,104.9,2.577320,158,112.0,2.724138,121.0792,64.2564,58.0,jhs


In [4]:
#take out the "study" variable
study = method3_df['study']
method3_df = method3_df.drop(columns = ['study']) 

method3_df.shape

(3771, 11)

In [5]:
# check if there are any missing values
method3_df.isnull().sum()
# check variable types
method3_df.dtypes



bmi          float64
hba1c        float64
dmagediag    float64
homa2b       float64
homa2ir      float64
tgl            int64
ldlc         float64
ratio_th     float64
sbp          float64
dbp          float64
hdlc         float64
dtype: object

In [6]:
#run kmeans clustering to create the TRUE labels
#standardize the data
scaler = StandardScaler()
method3_df_scaled = scaler.fit_transform(method3_df)

method3_df_scaled = pd.DataFrame(method3_df_scaled, columns=method3_df.columns)



In [7]:
method3_df_scaled.head()

,bmi,hba1c,dmagediag,homa2b,homa2ir,tgl,ldlc,ratio_th,sbp,dbp,hdlc
0,-0.032257,0.300357,0.645757,-0.220785,-0.458673,-0.242366,-0.774925,-0.090932,-0.326612,-0.636184,-0.668157
1,-0.832829,-0.048265,0.489212,-0.091397,-0.560173,-0.512939,-0.830633,-0.511314,-0.326612,-0.692720,0.252415
2,-0.408438,1.171911,-0.841426,-0.957942,-0.467753,-0.324714,0.562068,-0.033657,1.987405,1.625245,-0.975014
3,1.420073,0.300357,0.254393,-0.254023,-0.364617,-0.783512,-0.384969,-0.742123,0.264201,-0.353505,0.866129
4,-0.608743,-0.048265,1.193667,-0.194670,-0.324213,0.157612,0.032841,-0.250570,-0.228143,-0.749256,0.789414


In [8]:
# run kmeans and get cluster labels from the five variable method (method 3)
kmeans = KMeans(
    init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57
)

# select five variables to cluster
var_m3 = ['bmi', 'hba1c', 'dmagediag','homa2b','homa2ir']
cluster_m3 = method3_df_scaled[var_m3]

kmeans = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans.fit(cluster_m3)

KMeans(init='random', n_clusters=4, n_init=10, random_state=57)

In [9]:
# summarize the cluster labels 
method3_df_cluster = method3_df
method3_df_cluster['cluster'] = kmeans.labels_
method3_df_cluster['cluster'].value_counts()


cluster
0    1766
1    1481
3     290
2     234
Name: count, dtype: int64

In [10]:
#summarize variables by cluster
method3_df_cluster.groupby('cluster').mean()

,bmi,hba1c,dmagediag,homa2b,homa2ir,tgl,ldlc,ratio_th,sbp,dbp,hdlc
cluster,,,,,,,,,,,
0,29.624151,6.269762,70.743322,86.821631,2.107629,133.261608,106.051076,2.960120,127.687002,70.621010,50.251076
1,38.086894,6.384085,51.314338,133.598042,3.866042,155.496286,117.407562,3.903651,122.223853,80.640832,44.889804
2,34.958178,9.900000,55.524927,52.536325,3.077110,149.948718,116.474359,3.587225,126.418088,80.898582,47.239316
3,33.839130,6.470690,65.299499,323.935517,9.741132,153.713793,101.668966,3.860884,125.934720,71.448641,47.013793


In [12]:
# remane the cluster labels
method3_df_cluster['cluster'] = method3_df_cluster['cluster'].replace({0: 'MARD', 1: 'MOD', 2: 'SIDD', 3: 'SIRD'})
method3_df_cluster['cluster'].value_counts()


cluster
MARD    1766
MOD     1481
SIRD     290
SIDD     234
Name: count, dtype: int64

In [13]:
#summarize variables by cluster
method3_df_cluster.groupby('cluster').mean()

,bmi,hba1c,dmagediag,homa2b,homa2ir,tgl,ldlc,ratio_th,sbp,dbp,hdlc
cluster,,,,,,,,,,,
MARD,29.624151,6.269762,70.743322,86.821631,2.107629,133.261608,106.051076,2.960120,127.687002,70.621010,50.251076
MOD,38.086894,6.384085,51.314338,133.598042,3.866042,155.496286,117.407562,3.903651,122.223853,80.640832,44.889804
SIDD,34.958178,9.900000,55.524927,52.536325,3.077110,149.948718,116.474359,3.587225,126.418088,80.898582,47.239316
SIRD,33.839130,6.470690,65.299499,323.935517,9.741132,153.713793,101.668966,3.860884,125.934720,71.448641,47.013793


### NOW we will conduct sensitivity analysis 

In [14]:
# add the study variable back
method3_df_cluster['study'] = study


In [15]:
method3_df_cluster.head()


,bmi,hba1c,dmagediag,homa2b,homa2ir,tgl,ldlc,ratio_th,sbp,dbp,hdlc,cluster,study
0,33.37665,6.9,70.0,102.7,2.217295,124,83.0,3.179487,119.2454,65.9166,39.0,MARD,jhs
1,27.76165,6.5,68.0,113.6,1.945525,101,81.0,1.980392,119.2454,65.0865,51.0,MARD,jhs
2,30.73822,7.9,51.0,40.6,2.192982,117,131.0,3.342857,162.3397,99.1206,35.0,SIDD,jhs
3,43.56290,6.9,65.0,99.9,2.469136,78,97.0,1.322034,130.2482,70.0671,59.0,MOD,jhs
4,29.33333,6.5,77.0,104.9,2.577320,158,112.0,2.724138,121.0792,64.2564,58.0,MARD,jhs


In [29]:
# sample size by study
method3_df_cluster['study'].value_counts()

study
aric      923
dppos     860
dpp       685
jhs       660
cardia    366
mesa      277
Name: count, dtype: int64

In [28]:
# sample size by study and cluster
method3_df_cluster.groupby(['study', 'cluster']).size()


study   cluster
aric    MARD       854
        MOD         32
        SIDD        27
        SIRD        10
cardia  MARD        46
        MOD        255
        SIDD        59
        SIRD         6
dpp     MARD       175
        MOD        486
        SIDD        10
        SIRD        14
dppos   MARD       298
        MOD        495
        SIDD         3
        SIRD        64
jhs     MARD       319
        MOD        198
        SIDD       129
        SIRD        14
mesa    MARD        74
        MOD         15
        SIDD         6
        SIRD       182
dtype: int64

#### The Adjusted RAND Index method 
##### Note: sample sizes need to adjusted to match the reduced dataset when one study site is removed.  
##### Method 1: original label predictde from original dataset and then reduced to match the sample size with the leave-one-out study 

In [27]:
# Step 1: Create a list of unique study sites
study_sites = method3_df_cluster['study'].unique()

# Step 2: Iterate over each study site
for study_site in study_sites:
    # Step 2a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    # step 2b: standardize the data
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.fit_transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    # Step 2c: Run k-means clustering on the modified dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)

    kmeans = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans.fit(cluster_m3)

    # Step 2d: Compare cluster centers with the original cluster centers
    original_centers = kmeans.cluster_centers_
    excluded_centers = kmeans_excluded.cluster_centers_
    
    # Assess if any study site drives the clustering results using ARIs and Jaccard similarity
    # Step 2e: Calculate the Adjusted Rand Index (ARI) and Jaccard similarity
    from sklearn.metrics import adjusted_rand_score
    from sklearn.metrics import jaccard_score
    # remove the study site from the original dataset to run ARIs
    kmeans.labels_ = kmeans.labels_[method3_df_cluster['study'] != study_site]
    ARI = adjusted_rand_score(kmeans.labels_, kmeans_excluded.labels_)

    # Step 2f: Print the results
    print(f"Study site removed: {study_site}")
    print(f"ARI: {ARI}")
    print("\n")



Study site removed: jhs
ARI: 0.9706479621653834


Study site removed: dpp
ARI: 0.8977688266654225


Study site removed: dppos
ARI: 0.9518934390880112


Study site removed: aric
ARI: 0.5360172556892191


Study site removed: cardia
ARI: 0.9579487688813936


Study site removed: mesa
ARI: 0.6106109712104782




##### Method 2: use the new cluster centers to predict new lables in the original dataset. 

In [34]:
from sklearn.metrics import adjusted_rand_score


# Standardize the entire dataset first for the initial clustering
scaler = StandardScaler()
cluster_m3 = scaler.fit_transform(method3_df_cluster[var_m3])
cluster_m3 = pd.DataFrame(cluster_m3, columns=var_m3)

# Perform initial clustering on the full dataset
kmeans_full = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans_full.fit(cluster_m3)
original_labels = kmeans_full.labels_

# Store the ARI scores
ari_scores = []

for study_site in study_sites:
    # Step 1a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    
    # Step 1b: Standardize the data for the reduced dataset
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    
    # Step 1c: Run k-means clustering on the reduced dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)
    
    # Predict the labels for the full dataset using the cluster centers from the reduced dataset
    new_labels = kmeans_excluded.predict(cluster_m3)
    
    # Step 1d: Calculate the ARI between the original labels and the new labels
    ari = adjusted_rand_score(original_labels, new_labels)
    ari_scores.append(ari)
    
    # Print or store the comparison results
    print(f'When study {study_site} is removed:')
    print(f'ARI: {ari:.4f}')
    print()

# Summary of results
print("ARI Scores for each removed cohort:")
print(ari_scores)

When study jhs is removed:
ARI: 0.5443

When study dpp is removed:
ARI: 0.8756

When study dppos is removed:
ARI: 0.9044

When study aric is removed:
ARI: 0.5725

When study cardia is removed:
ARI: 0.9421

When study mesa is removed:
ARI: 0.5385

ARI Scores for each removed cohort:
[0.5443361625779876, 0.875554882417821, 0.9043842247477216, 0.5724792504286806, 0.9421444607070157, 0.5384714494760218]


#### The Cluster Center Stability Method 

##### Method 1


In [26]:
from scipy.spatial.distance import cdist

# Store the distances between cluster centers
center_distances = []

for study_site in study_sites:
    # Step 1a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    # step 1b: standardize the data
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.fit_transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    # Step 1c: Run k-means clustering on the modified dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)

    kmeans = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans.fit(cluster_m3)

    # Step 1d: Compare cluster centers with the original cluster centers
    original_centers = kmeans.cluster_centers_
    excluded_centers = kmeans_excluded.cluster_centers_

    # Step 1e: Calculate the distances between the cluster centers
    distances = cdist(original_centers, excluded_centers, 'euclidean')
       # Find the minimum distance for each cluster
    min_distances = np.min(distances, axis=1)
    center_distances.append(min_distances)
    
    # Print or store the comparison results
    print(f'When study {study_site} is removed:')
    print(f'Minimum distances between cluster centers: {min_distances}')
    print()

# Summary of results
print("Distances between cluster centers for each removed cohort:")
print(center_distances)


When study jhs is removed:
Minimum distances between cluster centers: [0.06507366 0.09501525 0.63563496 0.24966512]

When study dpp is removed:
Minimum distances between cluster centers: [0.06842183 0.11330323 0.12819198 0.18856308]

When study dppos is removed:
Minimum distances between cluster centers: [0.1125499  0.14554766 0.18290232 0.28171354]

When study aric is removed:
Minimum distances between cluster centers: [0.2572692  0.21223765 0.28202863 0.59069134]

When study cardia is removed:
Minimum distances between cluster centers: [0.06821373 0.12820293 0.27903413 0.0770222 ]

When study mesa is removed:
Minimum distances between cluster centers: [0.2027705  0.50647353 0.21177333 1.71428452]

Distances between cluster centers for each removed cohort:
[array([0.06507366, 0.09501525, 0.63563496, 0.24966512]), array([0.06842183, 0.11330323, 0.12819198, 0.18856308]), array([0.1125499 , 0.14554766, 0.18290232, 0.28171354]), array([0.2572692 , 0.21223765, 0.28202863, 0.59069134]), arr

##### Method 2

In [35]:
from scipy.spatial.distance import cdist

# Assuming method3_df_cluster is your original dataset and var_m3 contains the feature columns
# Assuming study_sites is a list of unique study sites

# Standardize the entire dataset first for the initial clustering
scaler = StandardScaler()
cluster_m3 = scaler.fit_transform(method3_df_cluster[var_m3])
cluster_m3 = pd.DataFrame(cluster_m3, columns=var_m3)

# Perform initial clustering on the full dataset
kmeans_full = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans_full.fit(cluster_m3)
original_centers = kmeans_full.cluster_centers_

# Store the distances between cluster centers
center_distances = []

for study_site in study_sites:
    # Step 1a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    
    # Step 1b: Standardize the data for the reduced dataset
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    
    # Step 1c: Run k-means clustering on the reduced dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)
    excluded_centers = kmeans_excluded.cluster_centers_

    # Step 1d: Calculate the distances between the original and excluded cluster centers
    distances = cdist(original_centers, excluded_centers, 'euclidean')
    
    # Find the minimum distance for each cluster
    min_distances = np.min(distances, axis=1)
    center_distances.append(min_distances)
    
    # Print or store the comparison results
    print(f'When study {study_site} is removed:')
    print(f'Minimum distances between cluster centers: {min_distances}')
    print()

# Summary of results
print("Distances between cluster centers for each removed cohort:")
print(center_distances)

When study jhs is removed:
Minimum distances between cluster centers: [0.23891254 0.81046867 3.20500304 0.13447452]

When study dpp is removed:
Minimum distances between cluster centers: [0.13991978 0.19356576 0.14622828 0.04198287]

When study dppos is removed:
Minimum distances between cluster centers: [0.13802243 0.24620865 0.17028364 0.34742316]

When study aric is removed:
Minimum distances between cluster centers: [0.78564242 0.50099177 0.34682706 0.15044379]

When study cardia is removed:
Minimum distances between cluster centers: [0.03319599 0.1562262  0.46905465 0.12087555]

When study mesa is removed:
Minimum distances between cluster centers: [0.22140273 0.94381023 0.11676106 2.86050974]

Distances between cluster centers for each removed cohort:
[array([0.23891254, 0.81046867, 3.20500304, 0.13447452]), array([0.13991978, 0.19356576, 0.14622828, 0.04198287]), array([0.13802243, 0.24620865, 0.17028364, 0.34742316]), array([0.78564242, 0.50099177, 0.34682706, 0.15044379]), arr

In [36]:
import numpy as np

# Collect all distances in a single list
all_distances = [
    [0.23891254, 0.81046867, 3.20500304, 0.13447452],
    [0.13991978, 0.19356576, 0.14622828, 0.04198287],
    [0.13802243, 0.24620865, 0.17028364, 0.34742316],
    [0.78564242, 0.50099177, 0.34682706, 0.15044379],
    [0.03319599, 0.1562262, 0.46905465, 0.12087555],
    [0.22140273, 0.94381023, 0.11676106, 2.86050974]
]

# Convert to a numpy array for easier calculations
all_distances = np.array(all_distances)

# Calculate mean and standard deviation for each cluster
mean_distances = np.mean(all_distances, axis=0)
std_distances = np.std(all_distances, axis=0)

print("Mean distances:", mean_distances)
print("Standard deviation of distances:", std_distances)

Mean distances: [0.25951598 0.47521188 0.74235962 0.60928494]
Standard deviation of distances: [0.24459859 0.30718209 1.10828481 1.01101766]


##### Method 3: relative shifts 


In [40]:
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


# Standardize the entire dataset first for the initial clustering
scaler = StandardScaler()
cluster_m3 = scaler.fit_transform(method3_df_cluster[var_m3])
cluster_m3 = pd.DataFrame(cluster_m3, columns=var_m3)

# Perform initial clustering on the full dataset
kmeans_full = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans_full.fit(cluster_m3)
original_centers = kmeans_full.cluster_centers_

# Print the original cluster centers
print("Original Cluster Centers:")
print(original_centers)
print()

# Calculate the magnitude of the original cluster centers
original_magnitudes = np.linalg.norm(original_centers, axis=1)

# Calculate the magnitude of the original cluster centers
original_magnitudes = np.linalg.norm(original_centers, axis=1)

# Store the relative distances between cluster centers
relative_center_distances = []

for study_site in study_sites:
    # Step 1a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    
    # Step 1b: Standardize the data for the reduced dataset
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    
    # Step 1c: Run k-means clustering on the reduced dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)
    excluded_centers = kmeans_excluded.cluster_centers_

    # Step 1d: Calculate the distances between the original and excluded cluster centers
    distances = cdist(original_centers, excluded_centers, 'euclidean')
    
    # Find the minimum distance for each cluster
    min_distances = np.min(distances, axis=1)
    
    # Calculate the relative distances
    relative_distances = min_distances / original_magnitudes
    relative_center_distances.append(relative_distances)
    
    # Print or store the comparison results
    print(f'When study {study_site} is removed:')
    print(f'Relative distances between cluster centers: {relative_distances}')
    print()

# Summary of results
print("Relative distances between cluster centers for each removed cohort:")
print(relative_center_distances)

Original Cluster Centers:
[[-0.56726754 -0.24884576  0.70379475 -0.41030861 -0.50028126]
 [ 0.63931782 -0.14929098 -0.81682184  0.14598928  0.15709501]
 [ 0.19323348  2.91501966 -0.48724723 -0.81625191 -0.13755256]
 [ 0.03154944 -0.07492061  0.28017616  2.40201897  2.34545022]]

When study jhs is removed:
Relative distances between cluster centers: [0.20972753 0.75767746 1.0421864  0.03990511]

When study dpp is removed:
Relative distances between cluster centers: [0.1228275  0.18095754 0.04754976 0.01245835]

When study dppos is removed:
Relative distances between cluster centers: [0.12116192 0.23017144 0.05537196 0.10309729]

When study aric is removed:
Relative distances between cluster centers: [0.68967013 0.46835885 0.11277944 0.04464396]

When study cardia is removed:
Relative distances between cluster centers: [0.02914085 0.14605015 0.15252478 0.03586963]

When study mesa is removed:
Relative distances between cluster centers: [0.19435667 0.8823336  0.03796776 0.84885186]

Relat

#### The Adjusted Mutual Information (AMI) method

#### Method 2

In [38]:
from sklearn.metrics import adjusted_mutual_info_score


# Standardize the entire dataset first for the initial clustering
scaler = StandardScaler()
cluster_m3 = scaler.fit_transform(method3_df_cluster[var_m3])
cluster_m3 = pd.DataFrame(cluster_m3, columns=var_m3)

# Perform initial clustering on the full dataset
kmeans_full = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans_full.fit(cluster_m3)
original_labels = kmeans_full.labels_

# Store the AMI scores
ami_scores = []

for study_site in study_sites:
    # Step 1a: Exclude the current study site from the dataset
    excluded_dataset = method3_df_cluster[method3_df_cluster['study'] != study_site]
    
    # Step 1b: Standardize the data
    excluded_cluster_m3 = excluded_dataset[var_m3]
    excluded_cluster_m3 = scaler.transform(excluded_cluster_m3)
    excluded_cluster_m3 = pd.DataFrame(excluded_cluster_m3, columns=var_m3)
    
    # Step 1c: Run k-means clustering on the modified dataset
    kmeans_excluded = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_excluded.fit(excluded_cluster_m3)
    excluded_centers = kmeans_excluded.cluster_centers_

    # Predict the labels for the entire original dataset using the cluster centers from the reduced dataset
    new_labels = kmeans_excluded.predict(cluster_m3)
    
    # Step 1d: Calculate the AMI between the original labels and the new labels
    ami = adjusted_mutual_info_score(original_labels, new_labels)
    ami_scores.append(ami)
    
    # Print or store the comparison results
    print(f'When study {study_site} is removed:')
    print(f'AMI: {ami:.4f}')
    print()

# Summary of results
print("AMI Scores for each removed cohort:")
print(ami_scores)

When study jhs is removed:
AMI: 0.5574

When study dpp is removed:
AMI: 0.8650

When study dppos is removed:
AMI: 0.8669

When study aric is removed:
AMI: 0.6594

When study cardia is removed:
AMI: 0.8964

When study mesa is removed:
AMI: 0.5754

AMI Scores for each removed cohort:
[0.5574476620776241, 0.8650469655144875, 0.8668663012783755, 0.6594258449878654, 0.8963988937552566, 0.5754475337457299]


### New assess cluster stability

In [43]:
from sklearn.utils import resample

# Original unscaled data
original_data = method3_df_cluster[var_m3]

# Initialize the scaler
scaler = StandardScaler()

# Standardize the entire original dataset
cluster_m3 = scaler.fit_transform(original_data)

# Perform initial clustering on the full dataset
kmeans_full = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
kmeans_full.fit(cluster_m3)
original_labels = kmeans_full.labels_

# Number of bootstrap samples
n_bootstraps = 1000
bootstrap_labels = []

for i in range(n_bootstraps):
    # Resample from the original unscaled data
    bootstrap_sample = resample(original_data, random_state=i)
    
    # Scale the bootstrap sample
    bootstrap_sample_scaled = scaler.transform(bootstrap_sample)
    
    # Apply k-means clustering to the scaled bootstrap sample
    kmeans_bootstrap = KMeans(init="random", n_clusters=4, n_init=10, max_iter=300, random_state=57)
    kmeans_bootstrap.fit(bootstrap_sample_scaled)
    
    # Predict labels for the original scaled data using the cluster centers from the bootstrap sample
    bootstrap_labels_full = kmeans_bootstrap.predict(cluster_m3)
    
    # Store the labels
    bootstrap_labels.append(bootstrap_labels_full)

# Calculate the mean adjusted Rand index across bootstrap samples
ari_bootstrap = [adjusted_rand_score(original_labels, labels) for labels in bootstrap_labels]
mean_ari_bootstrap = np.mean(ari_bootstrap)
print(f'Mean ARI across bootstrap samples: {mean_ari_bootstrap}')

Mean ARI across bootstrap samples: 0.9483573987651209
